Help is available on "https://www.youtube.com/watch?v=eiELMN7QIT0"

* MoleculeNet paper: https://arxiv.org/abs/1703.00564

* Esol/Delaney examples: https://github.com/deepchem/deepchem/...
* GraphConv from scratch: https://github.com/deepchem/deepchem/...
* DeepChem book https://www.amazon.com/Deep-Learning-..

In [1]:
!pip install tensorflow-gpu==1.15.2

     |████████████████████████████████| 411.0MB 43kB/s 
     |████████████████████████████████| 51kB 6.9MB/s 
     |████████████████████████████████| 3.8MB 39.9MB/s 
     |████████████████████████████████| 512kB 39.7MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7542 sha256=f087d99b1faceefa495a9f76c2160141eba0ee69c5e0418624d0f5644a0dcbf0
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow 2.3.0 has requirement gast==0.3.3, but you'll have gast 0.2.2 which is incompatible.
ERROR: tensorflow 2.3.0 has requirement tensorboard<3,>=2.3.0, but you'll have tensorboard 1.15.0 which is incompatible.
ERROR: tensorflow 2.3.0 has requirement tensorflow-estimator<2.4.0,>=2.3.0, but you'll have tensorflow-estimator 1.15.1 which is incompatible.
ERROR: tensorflow-probability 0.11.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing i

In [2]:
import tensorflow as tf
tf.__version__

'1.15.2'

In [3]:
# takes 5-10 minutes
!wget -c https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
!chmod +x Miniconda3-latest-Linux-x86_64.sh
!time bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local
!conda install -y -c deepchem -c rdkit -c conda-forge -c omnia deepchem-gpu=2.3.0 python=3.6

--2020-11-29 20:30:17--  https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.continuum.io (repo.continuum.io)... 104.18.200.79, 104.18.201.79, 2606:4700::6812:c84f, ...
Connecting to repo.continuum.io (repo.continuum.io)|104.18.200.79|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh [following]
--2020-11-29 20:30:17--  https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.131.3, 104.16.130.3, 2606:4700::6810:8203, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.131.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 94235922 (90M) [application/x-sh]
Saving to: ‘Miniconda3-latest-Linux-x86_64.sh’

Miniconda3-latest-L 100%[===================>]  89.87M   233MB/s    in 0.4s    

2020-11-29 20:30:18 (233 MB/s) - ‘Miniconda3-latest-Linux

In [4]:
import sys
sys.path.append('/usr/local/lib/python3.6/site-packages/') 
import deepchem as dc
import numpy as np
from deepchem.models import GraphConvModel
import tensorflow as tf
from deepchem.molnet import load_delaney

/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [5]:
#Load Delaney dataset
delaney_tasks, delaney_datasets, transformers = load_delaney(
    featurizer='GraphConv', split='index')
delaney_tasks

Loading raw samples now.
shard_size: 8192
About to start loading CSV from /tmp/delaney-processed.csv
Loading shard 1 of size 8192.
Featurizing sample 0
Featurizing sample 1000
TIMING: featurizing shard 0 took 3.933 s
TIMING: dataset construction took 4.263 s
Loading dataset from disk.
TIMING: dataset construction took 0.338 s
Loading dataset from disk.
TIMING: dataset construction took 0.148 s
Loading dataset from disk.
TIMING: dataset construction took 0.257 s
Loading dataset from disk.
TIMING: dataset construction took 0.306 s
Loading dataset from disk.
TIMING: dataset construction took 0.041 s
Loading dataset from disk.
TIMING: dataset construction took 0.045 s
Loading dataset from disk.


['measured log solubility in mols per litre']

In [6]:
delaney_datasets

(<deepchem.data.datasets.DiskDataset at 0x7f8eea603080>,
 <deepchem.data.datasets.DiskDataset at 0x7f8eeaba1be0>)

In [7]:
transformers

In [11]:
train_dataset, valid_dataset, test_dataset = delaney_datasets
train_dataset.get_data_shape

<bound method DiskDataset.get_data_shape of <deepchem.data.datasets.DiskDataset object at 0x7f8eea603080>>

In [12]:
# Fit models
metric = dc.metrics.Metric(dc.metrics.pearson_r2_score, np.mean)

# Do setup required for tf/keras models
# Number of features on conv-mols
n_feat = 75
# Batch size of models
batch_size = 128
model = GraphConvModel(
    len(delaney_tasks), batch_size=batch_size, mode='regression')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.




In [13]:
# Fit trained model
model.fit(train_dataset, nb_epoch=20)

print("Evaluating model")
train_scores = model.evaluate(train_dataset, [metric], transformers)
valid_scores = model.evaluate(valid_dataset, [metric], transformers)

print("Train scores")
print(train_scores)

print("Validation scores")
print(valid_scores)






Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Evaluating model
computed_metrics: [0.9385940615697673]
computed_metrics: [0.8209050088156183]
Train scores
{'mean-pearson_r2_score': 0.9385940615697673}
Validation scores
{'mean-pearson_r2_score': 0.8209050088156183}
